In [16]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB_sum = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB = -class_prob * decision_prob * math.log(decision_prob, 2)
                entropy_DB_sum=entropy_DB_sum+entropy_DB
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.1:
    #     return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(total_rule/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    # random.seed(42)
    #random.seed(1)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof


In [17]:
from sklearn.neighbors import KNeighborsClassifier

def bestIndividual_KNN(hof,df):
    k =7
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/WDBC.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [18]:
from sklearn.model_selection import StratifiedKFold

def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.1, max_iter=900)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/WDBC.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier


def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/WDBC.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier(max_depth=6)
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [20]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=30
            n_gen=30
            # read dataframe from csv
            df = pd.read_csv('data/WDBC.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            for col in feature_columns:
                df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
            # get accuracy with all features
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue


Accuracy with all features: 	(0.001937984496124031,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg      	min      
0  	30    	0.0483831	0.0103627
1  	18    	0.0200716	0.00680272
2  	18    	0.0150248	0.00653595
3  	18    	0.00992899	0.00419287
4  	21    	0.00757692	0.00426439
5  	13    	0.0065373 	0.00419287
6  	12    	0.00541904	0.00419287
7  	13    	0.00452253	0.00203666
8  	19    	0.00445941	0.0020202 
9  	13    	0.00368757	0.00198807
10 	15    	0.00391906	0.00196464
11 	18    	0.00266821	0.00196464
12 	14    	0.00213314	0.00196464
13 	21    	0.00204847	0.00195695
14 	17    	0.00196998	0.00195695
15 	19    	0.00196273	0.00195312
16 	14    	0.00195995	0.00195312
17 	20    	0.00211362	0.00195312
18 	21    	0.00209212	0.00194553
19 	18    	0.00195598	0.00194553
20 	16    	0.00208989	0.00194553
21 	12    	0.00216299	0.00194553
22 	15    	0.00195308	0.00194553
23 	23    	0.00217221	0.00194553
24 	19    	0.00242125	0.00194553
25 	8     	0.00202363	0.00194553
26 	15    	0.00196057	0.00194553
27 	17    	0.00194761	0.00194553
28 	17    	0.00202048	0.00193798
29 	24    	0.002

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg   	min    
0  	30    	1.0332	1.00498
1  	20    	1.01764	1.00464
2  	15    	1.01184	1.00464
3  	22    	1.00938	1.0044 
4  	17    	1.00667	1.00433
5  	14    	1.00579	1.00413
6  	21    	1.00479	1.00218
7  	17    	1.00466	1.00218
8  	23    	1.00459	1.00207
9  	15    	1.0034 	1.00207
10 	16    	1.0029 	1.00207
11 	14    	1.00297	1.00203
12 	19    	1.00237	1.00203
13 	23    	1.00223	1.00203
14 	15    	1.00246	1.00203
15 	15    	1.00205	1.00203
16 	16    	1.00204	1.00202
17 	24    	1.00226	1.00202
18 	20    	1.0023 	1.002  
19 	13    	1.00247	1.002  
20 	20    	1.00217	1.002  
21 	23    	1.00229	1.002  
22 	21    	1.00251	1.002  
23 	16    	1.00201	1.00199
24 	20    	1.00228	1.00199
25 	18    	1.00207	1.00199
26 	23    	1.00237	1.00199
27 	14    	1.00206	1.00199
28 	21    	1.00199	1.00199
29 	18    	1.00199	1.00198
30 	14    	1.00213	1.00197
Top 1 solution: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
Fitness: 1.0019723865877712

--

...

gen	nevals	avg   	min    
0  	30    	3.5108	3.47835
1  	14    	3.49494	3.47835
2  	20    	3.48388	3.47829
3  	19    	3.47974	3.47831
4  	18    	3.47856	3.47831
5  	20    	3.47892	3.47829
6  	20    	3.47846	3.47828
7  	14    	3.47852	3.47829
8  	16    	3.47872	3.47829
9  	17    	3.4783 	3.47829
10 	27    	3.47844	3.47829
11 	18    	3.4783 	3.47828
12 	15    	3.4783 	3.47828
13 	16    	3.47858	3.47828
14 	17    	3.47843	3.47828
15 	21    	3.47836	3.47827
16 	16    	3.47835	3.47827
17 	18    	3.47828	3.47827
18 	19    	3.47834	3.47827
19 	18    	3.47854	3.47827
20 	17    	3.47834	3.47827
21 	19    	3.47834	3.47827
22 	19    	3.47834	3.47827
23 	19    	3.4784 	3.47827
24 	12    	3.47827	3.47827
25 	16    	3.47834	3.47827
26 	20    	3.47847	3.47827
27 	17    	3.47841	3.47827
28 	19    	3.47842	3.47827
29 	10    	3.47834	3.47827
30 	20    	3.47827	3.47827
Top 1 solution: [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
Fitness: 3.4782668789397215

--

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	30    	4.51489	4.48103
1  	23    	4.49854	4.4785 
2  	18    	4.48898	4.47849
3  	23    	4.4839 	4.47841
4  	11    	4.48144	4.47841
5  	19    	4.47949	4.47841
6  	17    	4.47902	4.47836
7  	19    	4.47856	4.47836
8  	20    	4.47862	4.47831
9  	22    	4.47837	4.47831
10 	15    	4.47857	4.47831
11 	20    	4.47853	4.47828
12 	25    	4.47841	4.47828
13 	17    	4.47831	4.47828
14 	23    	4.4783 	4.47828
15 	19    	4.47829	4.47828
16 	19    	4.47829	4.47828
17 	12    	4.4785 	4.47828
18 	18    	4.47829	4.47828
19 	16    	4.47836	4.47828
20 	15    	4.47859	4.47828
21 	17    	4.47858	4.47826
22 	21    	4.47842	4.47826
23 	18    	4.47857	4.47826
24 	18    	4.47848	4.47826
25 	18    	4.47827	4.47826
26 	17    	4.4784 	4.47826
27 	17    	4.47833	4.47826
28 	21    	4.4784 	4.47826
29 	24    	4.47833	4.47826
30 	18    	4.47849	4.47826
Top 1 solution: [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
Fitness: 4.478259308773746

-

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	30    	5.51184	5.47859
1  	21    	5.49005	5.47859
2  	15    	5.4852 	5.47859
3  	18    	5.48234	5.47859
4  	19    	5.48006	5.47845
5  	16    	5.47991	5.47843
6  	19    	5.4789 	5.47834
7  	18    	5.47885	5.47834
8  	18    	5.47912	5.47836
9  	21    	5.4787 	5.47835
10 	16    	5.47853	5.47833
11 	19    	5.4785 	5.47831
12 	14    	5.47835	5.47831
13 	14    	5.47855	5.47831
14 	16    	5.47878	5.47831
15 	19    	5.47848	5.47831
16 	16    	5.47854	5.47829
17 	17    	5.47838	5.47829
18 	13    	5.47864	5.47829
19 	11    	5.47866	5.47827
20 	19    	5.47829	5.47827
21 	15    	5.47828	5.47827
22 	21    	5.47873	5.47827
23 	18    	5.47834	5.47827
24 	20    	5.47834	5.47826
25 	16    	5.47834	5.47826
26 	23    	5.47883	5.47826
27 	17    	5.4784 	5.47826
28 	19    	5.4786 	5.47826
29 	20    	5.4784 	5.47826
30 	10    	5.47846	5.47826
Top 1 solution: [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
Fitness: 5.478259308773746

-